In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import pandas as pd

In [2]:
def merge_images(img1, img2):
    cloud_pixel = np.array([255, 255, 255])
    mask = np.all(img1 == cloud_pixel, axis=-1)
    img1[mask] = img2[mask]
    return img1


In [3]:
# Load and preprocess data
X_train = []
# Load labels
y_train = pd.read_csv("level_04/train_data_labels.csv", header=None).values.squeeze()

for i in range(500):
    img_path1 = f"level_04/train_data/field{i:03d}_sample0.png"
    img_path2 = f"level_04/train_data/field{i:03d}_sample1.png"
    img1 = np.asarray(Image.open(img_path1).convert('RGB'))
    img2 = np.asarray(Image.open(img_path2).convert('RGB'))
    merged_img = merge_images(img1, img2)
    X_train.append(merged_img)

X_train = np.array(X_train) / 255.0  # Normalize images to [0, 1]

In [4]:
# Define the CNN model for regression
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2)  # 2 outputs for x and y coordinates
])

# Compile model
model.compile(optimizer='adam', loss='mse')

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-10-20 17:14:14.508417: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-20 17:14:14.508528: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# Train model
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.01)


2023-10-20 17:14:14.787148: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-10-20 17:14:14.928403: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/100
50/50 [==============================] - 5s 93ms/step - loss: 3337.2385 - val_loss: 2265.7302
Epoch 2/100
 1/50 [..............................] - ETA: 4s - loss: 2084.1780

2023-10-20 17:14:19.714778: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 4s 81ms/step - loss: 2466.4873 - val_loss: 2315.8093
Epoch 3/100
50/50 [==============================] - 4s 81ms/step - loss: 1782.3167 - val_loss: 855.6254
Epoch 4/100
50/50 [==============================] - 4s 82ms/step - loss: 360.3517 - val_loss: 90.1714
Epoch 5/100
50/50 [==============================] - 4s 84ms/step - loss: 118.2976 - val_loss: 38.0881
Epoch 6/100
50/50 [==============================] - 4s 82ms/step - loss: 75.7252 - val_loss: 43.4508
Epoch 7/100
50/50 [==============================] - 4s 82ms/step - loss: 47.6353 - val_loss: 23.8240
Epoch 8/100
50/50 [==============================] - 4s 83ms/step - loss: 32.8196 - val_loss: 10.1775
Epoch 9/100
50/50 [==============================] - 4s 84ms/step - loss: 42.4224 - val_loss: 58.4562
Epoch 10/100
50/50 [==============================] - 4s 81ms/step - loss: 29.7817 - val_loss: 39.4335
Epoch 11/100
50/50 [==============================] - 4s 84ms/step - loss: 19.3718 -

In [6]:
merged_test_images = []
for i in range(1000):
    img_path1 = f"level_04/test_data/field{i:03d}_sample0.png"
    img_path2 = f"level_04/test_data/field{i:03d}_sample1.png"
    img1 = np.asarray(Image.open(img_path1).convert('RGB'))
    img2 = np.asarray(Image.open(img_path2).convert('RGB'))
    merged_img = merge_images(img1, img2)
    merged_test_images.append(merged_img)

test_data = np.array(merged_test_images) / 255.0  # Normalize to [0, 1]


In [7]:
predictions = model.predict(test_data)


2023-10-20 17:21:12.235678: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [8]:
np.savetxt("predictions.csv", predictions, delimiter=",", fmt='%d')
